#### Exercícios

- 1 - Teste diferentes regressores CV com a base house_prices
    - Aumente a complexidade da hipótese permitindo uma regressão polinomial, e deixe o CV decidir uma boa regularização!

In [74]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.linear_model import RidgeCV, LogisticRegressionCV

from sklearn.metrics import classification_report, r2_score, mean_squared_error

In [25]:
house_prices = pd.read_csv('house_prices.csv')
house_prices.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [4]:
print(house_prices.shape)
house_prices.info()

(2919, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 1

In [5]:
house_prices.isnull().sum().sort_values(ascending=False).head(40)

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
MasVnrType       1766
SalePrice        1459
FireplaceQu      1420
LotFrontage       486
GarageYrBlt       159
GarageFinish      159
GarageQual        159
GarageCond        159
GarageType        157
BsmtExposure       82
BsmtCond           82
BsmtQual           81
BsmtFinType2       80
BsmtFinType1       79
MasVnrArea         23
MSZoning            4
Functional          2
BsmtFullBath        2
Utilities           2
BsmtHalfBath        2
Electrical          1
TotalBsmtSF         1
BsmtUnfSF           1
KitchenQual         1
BsmtFinSF2          1
BsmtFinSF1          1
SaleType            1
GarageCars          1
Exterior2nd         1
Exterior1st         1
GarageArea          1
YrSold              0
Fireplaces          0
MoSold              0
MiscVal             0
SaleCondition       0
dtype: int64

In [37]:
# Como quase 50% do dataset contém dados faltantes para 'SalePrice', excluí linhas sem valores do modelo.
# As amostras foram salvas num novo dataset que poderá ser utilizado como novos dados de entrada do modelo para predição futuramente.
house_prices_filtered = house_prices.dropna(subset='SalePrice')
no_price = house_prices[house_prices['SalePrice'].isna()]

print(house_prices_filtered.shape)
no_price

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1460,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,NaN
1461,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,NaN
1462,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,NaN
1463,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,NaN
1464,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [38]:
house_prices_filtered.isnull().sum().sort_values(ascending=False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
Id                 0
dtype: int64

In [39]:
house_filtered = house_prices_filtered.drop(columns=['PoolQC',
                                                    'MiscFeature',
                                                    'Alley',
                                                    'Fence',
                                                    'MasVnrType',
                                                    'FireplaceQu',
                                                    'LotFrontage'],
                                            axis=1)

house_filtered

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500.0
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500.0
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500.0
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000.0
1456,1457,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000.0
1457,1458,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500.0
1458,1459,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125.0


In [40]:
house_filtered.isnull().sum().sort_values(ascending=False).head(15)

GarageType      81
GarageCond      81
GarageYrBlt     81
GarageFinish    81
GarageQual      81
BsmtFinType2    38
BsmtExposure    38
BsmtFinType1    37
BsmtCond        37
BsmtQual        37
MasVnrArea       8
Electrical       1
WoodDeckSF       0
PavedDrive       0
LowQualFinSF     0
dtype: int64

In [41]:
# Exclusão de linhas com cerca de <5% de dados faltantes por coluna
house_filtered.dropna(subset=['GarageType', 'GarageCond', 'GarageYrBlt', 'GarageFinish',
                              'GarageQual', 'BsmtFinType2', 'BsmtExposure','BsmtFinType1',
                              'BsmtCond', 'BsmtQual', 'MasVnrArea', 'Electrical'],
                      inplace=True)

house_filtered.shape

(1338, 74)

In [48]:
house_filtered_dummies = pd.get_dummies(house_filtered,
                                        drop_first=True,
                                        dtype=int)
corr = house_filtered_dummies.corr()
corr[(corr['SalePrice'] >=0.1) | (corr['SalePrice'] <= -0.1)]['SalePrice'].sort_values(ascending=False)


SalePrice            1.000000
OverallQual          0.783546
GrLivArea            0.711706
GarageCars           0.640154
GarageArea           0.607535
                       ...   
GarageType_Detchd   -0.399803
GarageFinish_Unf    -0.478025
BsmtQual_TA         -0.484357
KitchenQual_TA      -0.524120
ExterQual_TA        -0.585107
Name: SalePrice, Length: 95, dtype: float64

In [49]:
# Para simplificação do exercício, utilizarei somente as colunas com variáveis numéricas
corr2 = house_filtered.corr(numeric_only=True)
corr2['SalePrice'].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.783546
GrLivArea        0.711706
GarageCars       0.640154
GarageArea       0.607535
1stFlrSF         0.604714
TotalBsmtSF      0.602042
FullBath         0.569313
TotRmsAbvGrd     0.551821
YearBuilt        0.504297
YearRemodAdd     0.501435
GarageYrBlt      0.481730
MasVnrArea       0.465811
Fireplaces       0.445434
BsmtFinSF1       0.359677
OpenPorchSF      0.322786
2ndFlrSF         0.311354
WoodDeckSF       0.305983
HalfBath         0.258175
LotArea          0.254757
BsmtFullBath     0.209695
BsmtUnfSF        0.191689
BedroomAbvGr     0.169266
ScreenPorch      0.096624
PoolArea         0.091881
3SsnPorch        0.042159
MoSold           0.041310
LowQualFinSF    -0.009992
MiscVal         -0.016990
YrSold          -0.020451
BsmtHalfBath    -0.030175
Id              -0.030473
BsmtFinSF2      -0.031226
MSSubClass      -0.079599
OverallCond     -0.108627
KitchenAbvGr    -0.111408
EnclosedPorch   -0.127385
Name: SalePrice, dtype: float64

In [51]:
# Feature Selection usando resultado de correlação com a Target (valores > 0.1 ou <-0.1)
house_filtered = house_filtered[['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars',
                                 'GarageArea', '1stFlrSF', 'TotalBsmtSF', 'FullBath',
                                 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt',
                                 'MasVnrArea', 'Fireplaces', 'BsmtFinSF1', 'OpenPorchSF',
                                 '2ndFlrSF', 'WoodDeckSF', 'HalfBath', 'LotArea', 'BsmtFullBath',
                                 'BsmtUnfSF', 'BedroomAbvGr', 'OverallCond', 'KitchenAbvGr',
                                 'EnclosedPorch']]

house_filtered.head()

,SalePrice,OverallQual,GrLivArea,GarageCars,GarageArea,1stFlrSF,TotalBsmtSF,FullBath,TotRmsAbvGrd,YearBuilt,...,2ndFlrSF,WoodDeckSF,HalfBath,LotArea,BsmtFullBath,BsmtUnfSF,BedroomAbvGr,OverallCond,KitchenAbvGr,EnclosedPorch
0,208500.0,7,1710,2.0,548.0,856,856.0,2,8,2003,...,854,0,1,8450,1.0,150.0,3,5,1,0
1,181500.0,6,1262,2.0,460.0,1262,1262.0,2,6,1976,...,0,298,0,9600,0.0,284.0,3,8,1,0
2,223500.0,7,1786,2.0,608.0,920,920.0,2,6,2001,...,866,0,1,11250,1.0,434.0,3,5,1,0
3,140000.0,7,1717,3.0,642.0,961,756.0,1,7,1915,...,756,0,0,9550,1.0,540.0,3,5,1,272
4,250000.0,8,2198,3.0,836.0,1145,1145.0,2,9,2000,...,1053,192,1,14260,1.0,490.0,4,5,1,0


In [52]:
X = house_filtered.drop('SalePrice', axis=1)
y = house_filtered['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
ridge = RidgeCV(alphas=np.linspace(0.1, 1, 50), cv=kf)

ridge.fit(X_train, y_train)

RidgeCV(alphas=array([0.1       , 0.11836735, 0.13673469, 0.15510204, 0.17346939,
       0.19183673, 0.21020408, 0.22857143, 0.24693878, 0.26530612,
       0.28367347, 0.30204082, 0.32040816, 0.33877551, 0.35714286,
       0.3755102 , 0.39387755, 0.4122449 , 0.43061224, 0.44897959,
       0.46734694, 0.48571429, 0.50408163, 0.52244898, 0.54081633,
       0.55918367, 0.57755102, 0.59591837, 0.61428571, 0.63265306,
       0.65102041, 0.66938776, 0.6877551 , 0.70612245, 0.7244898 ,
       0.74285714, 0.76122449, 0.77959184, 0.79795918, 0.81632653,
       0.83469388, 0.85306122, 0.87142857, 0.88979592, 0.90816327,
       0.92653061, 0.94489796, 0.96326531, 0.98163265, 1.        ]),
        cv=KFold(n_splits=10, random_state=42, shuffle=True))

In [56]:
print(ridge.alpha_)
ridge.best_score_

1.0


0.7601257439067708

In [57]:
y_pred_train = ridge.predict(X_train)
y_pred_test = ridge.predict(X_test)

print('Métricas:')
results = {'Treino/Teste': ['y_pred_train', 'y_pred_test'],
           'R2': [r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test)],
           'MSE' : [mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)]}

pd.DataFrame(results).sort_values(by='R2', ascending=False)

Métricas:


,Treino/Teste,R2,MSE
1,y_pred_test,0.799693,8.571344e+08
0,y_pred_train,0.799285,1.342664e+09


- 2 - Aplique o logit com CV à base breast_cancer

In [68]:
breast_cancer = pd.read_csv('breast_cancer.csv')
breast_cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [61]:
print(breast_cancer.shape)
breast_cancer.info()

(569, 32)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se    

In [62]:
breast_cancer['diagnosis'].value_counts()

diagnosis
B    357
M    212
Name: count, dtype: int64

In [63]:
breast_cancer.isnull().sum()

id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

In [69]:
breast_cancer['diagnosis_encoded'] = breast_cancer['diagnosis'].map({'B': 0,
                                                                 'M': 1})
breast_cancer['diagnosis_encoded'].head()

0    1
1    1
2    1
3    1
4    1
Name: diagnosis_encoded, dtype: int64

In [70]:
corr = breast_cancer.corr(numeric_only=True)
corr['diagnosis_encoded'].sort_values(ascending=False)

diagnosis_encoded          1.000000
concave points_worst       0.793566
perimeter_worst            0.782914
concave points_mean        0.776614
radius_worst               0.776454
perimeter_mean             0.742636
area_worst                 0.733825
radius_mean                0.730029
area_mean                  0.708984
concavity_mean             0.696360
concavity_worst            0.659610
compactness_mean           0.596534
compactness_worst          0.590998
radius_se                  0.567134
perimeter_se               0.556141
area_se                    0.548236
texture_worst              0.456903
smoothness_worst           0.421465
symmetry_worst             0.416294
texture_mean               0.415185
concave points_se          0.408042
smoothness_mean            0.358560
symmetry_mean              0.330499
fractal_dimension_worst    0.323872
compactness_se             0.292999
concavity_se               0.253730
fractal_dimension_se       0.077972
id                         0

In [73]:
X = breast_cancer.drop(columns=['diagnosis', 'diagnosis_encoded'], axis=1)
y = breast_cancer['diagnosis_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

logit = LogisticRegressionCV(Cs=np.linspace(0.01, 10, 100),
                             scoring='f1',
                             cv=skf)

logit.fit(X_train, y_train)



LogisticRegressionCV(Cs=array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.71636364,  0.81727273,  0.91818182,
        1.01909091,  1.12      ,  1.22090909,  1.32181818,  1.42272727,
        1.52363636,  1.62454545,  1.72545455,  1.82636364,  1.92727273,
        2.02818182,  2.12909091,  2.23      ,  2.33090909,  2.43181818,
        2.53272727,  2.63363636,  2.73454545,  2.83545455,  2.93636364,
        3.03727...
        7.07363636,  7.17454545,  7.27545455,  7.37636364,  7.47727273,
        7.57818182,  7.67909091,  7.78      ,  7.88090909,  7.98181818,
        8.08272727,  8.18363636,  8.28454545,  8.38545455,  8.48636364,
        8.58727273,  8.68818182,  8.78909091,  8.89      ,  8.99090909,
        9.09181818,  9.19272727,  9.29363636,  9.39454545,  9.49545455,
        9.59636364,  9.69727273,  9.79818182,  9.89909091, 10.        ]),
                     cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
                     scoring='f1')

In [85]:
y_pred_train = logit.predict(X_train)
y_pred_test = logit.predict(X_test)

print('Classification Report Treino')
print(classification_report(y_train, y_pred_train))
print('\nClassification Report Teste')
print(classification_report(y_test, y_pred_test))

Classification Report Treino
              precision    recall  f1-score   support

           0       0.60      0.02      0.04       286
           1       0.37      0.98      0.54       169

    accuracy                           0.38       455
   macro avg       0.49      0.50      0.29       455
weighted avg       0.51      0.38      0.23       455


Classification Report Teste
              precision    recall  f1-score   support

           0       1.00      0.01      0.03        71
           1       0.38      1.00      0.55        43

    accuracy                           0.39       114
   macro avg       0.69      0.51      0.29       114
weighted avg       0.77      0.39      0.23       114

